**Название работы**: Исследование надёжности заёмщиков

Постановка задачи: Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Входные данные от банка — статистика о платёжеспособности клиентов.

 **Содержание**

    1. Загрузка датасета и анализ его содержимого
    2. Предобработка данных
    3. Проверка гипотез
    4. Общий вывод по проекту


# Загрузка датасета и анализ его содержимого

In [1]:
# Первым делом произведем загрузку необходимых библиотек

import pandas as pd
from pymystem3 import Mystem

In [2]:
# Загружаем исходный датасет в переменную data

data = pd.read_csv('data.csv')

# Выводим на экран датасет для визуального ознакомления

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Сразу бросается в глаза наличие артефактов в датасете: использование разного формата записи категориальных признаков (в столбце education одно и то же значение записано неодинаково), отрицательные числа в значениях рабочего стажа и др. В дальнейшем, после детального изучения данных, необходимо будет принять меры по устранению данных проблем.

In [3]:
# Выводим информацию о содержимом датафрейма

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Изучение содержимого столбцов датасета

In [4]:
# Для оценки состава датасета произведем подсчет уникальных признаков

for col in data:
    col
    print('Содержание столбца "{}":'.format(col))
    print(data[col].value_counts())
    print()
    
# Для числовых признаков выведем максимум и минимум с целью поиска артефактов

    if data[col].dtype != object:
        print('Максимальное значение столбца', col, ':', data[col].max())
        print('Минимальное значение столбца', col, ':', data[col].min())
    
    print('________________________________________________________')

Содержание столбца "children":
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Максимальное значение столбца children : 20
Минимальное значение столбца children : -1
________________________________________________________
Содержание столбца "days_employed":
-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

Максимальное значение столбца days_employed : 401755.40047533
Минимальное значение столбца days_employed : -18388.949900568383
________________________________________________________
Содержание столбца "dob_years":
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
2

## Выводы по результатам изучения датасета
 
 Файл содержит 21525 строк и 12 столбцов. Ниже представлено описание каждого столбца датасета.

1. **children** - количество детей заемщика, формат данных int, пропусков нет
    - больше всего клиентов банка не имеет детей;
    - количество детей в столбце children доходит до 20 (в принципе, возможно, но вероятность такой многодетности достаточно мала, а здесь 76 случаев! В рамках данного проекта оставляем как есть, так как дальше данные будут разбиты на категории. В обратном случае необходимо было бы заменить данное значение модой);
    - присутствует артефакт в виде значения -1 (скорее всего, ошибка при записи данных. Нужно заменить абсолютным значением признака);
2. **days_employed** - общий трудовой стаж клиента в днях, формат float, 2174 пропуска
    - максимальное значение столбца days_employed - 401755.4, что соответствует 1100 годам трудового стажа - артефакт. В данной задаче это не имеет значения, но в случае необходимости его можно исправить путем замены на медианное значение стажа по соответствующему возрасту клиента; 
    - в наличии отрицательный трудовой стаж - можно заменить на абсолютное значение признака;
    - пропуски в данных нужно заменить медианным значением;
3. **dob_years** - возраст клиента в годах, формат int, пропусков нет
    - имеются нулевые значения, что нелогично в данном столбце. Следует заменить на средний показатель возраста;
4. **education** - образование клиента, формат object (категориальный признак), пропусков нет
    - содержимое данного столбца нужно перевести в нижний регистр с помощью метода lower().
5. **education_id** - образование клиента в виде чисел от 0 до 4, формат int, пропусков нет
    - столбец не требует преобразований;
6. **family_status** - семейное положение клиента в формате object, пропусков нет 
    - столбец не требует преобразований; 
7. **family_status_id** - идентификатор семейного положения в формате int, пропусков нет
    - столбец не требует преобразований;
8. **gender** - пол клиента, формат object, пропусков нет
    - среди клиентов пришелец - его пол XNA! Данный артефакт можно заменить на показатель моды по столбцу;
9. **income_type** - информация о профессии клиента, формат object, пропусков нет
    - столбец не требует преобразований;
10. **debt** - информация о наличии задолжности клиента по выплате кредита, где 0 - отсутствие задолженности, 1 - ее наличие, формат int, пропусков нет
    - столбец не требует преобразований;
11. **total_income** - уровень ежемесячного дохода клиента, float, 2174 пропуска
    - необходимо заполнить пропуски медианным значением;
12. **purpose** - причина взятия кредита клиентом, формат object 
    - для удобства анализа данных необходимо произвести лемматизацию данных столбца.

# Предобработка данных

## Обработка отрицательных значений столбцов children

In [5]:
# Заменим все заничения столбца children на их абсолютное значение

data['children'] = abs(data['children'])

# Для проверки сгруппируем количество клиентов по количеству детей

print(data.loc[:,'children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Таким образом, исправлена аномалия со значением количества детей, равное -1, которая искажала общую картину.

## Замена нулей в показаниях столбца dob_years

In [6]:
# Рассчитываем показание среднего возраста клиента

dob_years_mean = int(data['dob_years'].mean())

In [7]:
# Заменяем нулевые показания столбца на среднее значение

for i in range(len(data)):
    if data.loc[i, 'dob_years'] == 0:
        data.loc[i, 'dob_years'] = dob_years_mean

In [8]:
# Проверяем минимальное значение столбца

data['dob_years'].min()

19

Нулевые значения возраста клиентов исправлены на среднее значение. 

## Обработка пропусков и отрицательных значений в стобце days_employed 

In [9]:
# Заменим все заничения столбца days_employed на их абсолютное значение

data['days_employed'] = abs(data['days_employed'])

# Для проверки вызовем минимальное значение столбца

print('Минимальное значение столбца days_employed равно:', data.loc[:,'days_employed'].min())

Минимальное значение столбца days_employed равно: 24.14163324048118


In [10]:
# Выведем начальное количество пропусков в столбце days_employed

print('Количество пропусков в столбце days_employed до коррекции:',data['days_employed'].isna().sum())

# Заменим пропущенные значения данного столбца на значение медианы по столбцу

data['days_employed'] = data['days_employed'].fillna(data.groupby(['education_id', 'income_type'])['days_employed'].transform('median'))
        
# Выведем конечное количество пропусков в столбце days_employed

print('Количество пропусков в столбце days_employed после их замены:', data['days_employed'].isna().sum())

Количество пропусков в столбце days_employed до коррекции: 2174
Количество пропусков в столбце days_employed после их замены: 0


Данные в столбце days_employed были приведены к их абсолютному значению. Пропуски заменены медианными значениями столбца (значение рассчитано по группам, которые были сформированы по значениям столбцов education_id и income_type). 

## Заполнение пропусков в столбце  total_income

In [11]:
# Выведем начальное количество пропусков в столбце total_income

print('Количество пропусков в столбце total_income до коррекции:', data['total_income'].isnull().sum())

# Заменяем пропуски на медианное значение дохода по столбцам education_id и income_type

data['total_income'] = data['total_income'].fillna(data.groupby(['education_id', 'income_type'])['total_income'].transform('median'))

# Выведем начальное количество пропусков в столбце total_income

print('Количество пропусков в столбце total_income после из замены на медиану:', data['total_income'].isnull().sum())

Количество пропусков в столбце total_income до коррекции: 2174
Количество пропусков в столбце total_income после из замены на медиану: 0


**Промежуточный вывод**
    На данном этапе в столбцах days_employed и total_income были заменены пропуски на преобразованные данные из столбца dob_years и медианное значение дохода соответственно. Данные преобразования позволят при необходимости смело использовать указанные столбцы для решения поставленных задач.

## Перевод содержимого столбца education в нижний регистр


In [15]:
# Переведем все значения столбца education в нижний регистр с помощью метода lower()

data['education'] = data['education'].str.lower()

# Для проверки посмотрим уникальные значения данного столбца

print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


Содержимое столбца education переведено в нижний регистр, что позволило увидеть истинный состав признаков.

## Замена артефакта в столбце gender

In [16]:
# Для начала определим превалирующий пол среди клиентов

data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [17]:
# Заменяем артефакт на определитель женского рода

data['gender'] = data['gender'].replace('XNA', 'F')

# Проверяем содержимое столбца

data['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Столбец gender избавлен от артефактного значения, что позволит избежать ошибок при использовании данных.

## Замена типа данных cтолбца days_employed

In [20]:
# Проверим текущий формат столбца days_employed
print('Тип данных столбца days_employed до форматирования:', data['days_employed'].dtype)

# Заменим тип данных столбца days_employed на int
data['days_employed'] = data['days_employed'].astype('int')

# Проверим результат выполненной работы
print('Тип данных столбца days_employed после форматирования:', data['days_employed'].dtype)

Тип данных столбца days_employed до форматирования: float64
Тип данных столбца days_employed после форматирования: int64


## Обработка дубликатов

In [21]:
# Выводим список всех продублированных значений во всех столбцах для сравнения

for col in data:
    print(data[col].duplicated().sum())
    print('Name of column', col)
    print(data[col].value_counts())
    print()

21518
Name of column children
0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

12437
Name of column days_employed
1613      827
365025    345
1670      301
1556      243
1454      192
         ... 
332955      1
351394      1
3256        1
5307        1
2049        1
Name: days_employed, Length: 9088, dtype: int64

21468
Name of column dob_years
35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, 

Предоставленный набор данных не позволяет выяснить определить продублированные записи о заемщике, так как нет индивидуального идентификатора (ФИО, номер паспорта, номер телефона, личный id и др.), по которому можно определить, что один и тот же человек записан 2 раза. В связи с этим нецелесообразно удалять одинаковые элементы, так как это приведет к потере данных, которые на самом деле могут не являться копиями друг друга. 

## Лемматизация и категоризация

In [23]:
# Сохраним инструмент для лемматизации в отдельную переменную

m = Mystem()

In [24]:
# Создаем функцию для лемматизации значений столбца purpose в датасете, 
# категоризируем значения и пишем результат в столбец purpose_category

def purpose_category(purpose):
    
    lemmas = m.lemmatize(purpose)
    
    if 'свадьба' in lemmas:
            return 'свадьба'
    
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
            return 'жилье/недвижимость'
    
    if 'автомобиль' in lemmas:
            return 'автомобиль'
    
    if 'образование' in lemmas:
            return 'образование'

In [25]:
# Применяем полученную функцию к датасету

data['purpose_category'] = data['purpose'].apply(purpose_category)     
data['purpose_category'].value_counts()

жилье/недвижимость    10840
автомобиль             4315
образование            4022
свадьба                2348
Name: purpose_category, dtype: int64

**Вывод по разделу** Данные были очищены от пропусков методом заполнения медианным значением или модой и приведены в такую форму, чтобы в дальнейшем можно было использовать имеющуюся информацию для получения требуемых нам знаний. 

Произведена лемматизация содержимого столбца purpose. Выявлены категории в столбце purpose, что позволило узнать, с какими основными намерениями клиенты берут деньги (4 категории, присвоены каждому клиенту и записаны в новый столбец purpose_category). 

В имеющихся данных можно обнаружить 2 словаря: 
    1. family_status_id - family_status (устанавливает численный эквивалент каждой строке с семейным статусом клиента);
    2. education_id - education (аналогичный эффект, но для строк с образованием). 

## Категоризация данных столбца total_income

In [26]:
# Создаем функцию категоризации по уровню дохода и записи категорий в новый столбец

def income_category(total_income):
    
    if  total_income  < 100000:
            return 'До 100 тыс'
            
    if  100000 <= total_income < 200000:
            return 'От 100 до 200 тыс'
            
    if  total_income >= 200000:
            return 'От 200 и выше тыс'

In [27]:
# Вызываем функцию категоризации и выводим результат

data['income_category'] = data['total_income'].apply(income_category)
data['income_category'].value_counts()

От 100 до 200 тыс    11807
От 200 и выше тыс     5255
До 100 тыс            4463
Name: income_category, dtype: int64

Данные в столбце total_income разделены на 3 категории для большей наглядности. Полученные категории присвоены каждому клиенту, результат записан в новый столбец income_category.

## Категоризация данных столбца dob_years

In [5]:
# Создаем функцию категоризации по трудовому стажу в годах и записи категорий в новый столбец dob_years_category

def dob_years_category(dob_years):

    if  dob_years  < 30:
        return 'До 30 лет'
            
    if  30 <= dob_years < 45 :
        return 'От 30 до 45 лет'
            
    if  dob_years >= 45:
        return 'От 45 лет'

In [29]:
# Вызываем функцию категоризации и выводим результат

data['dob_years_category'] = data['dob_years'].apply(dob_years_category)           
data['dob_years_category'].value_counts()

От 45 лет          9694
От 30 до 45 лет    8648
До 30 лет          3183
Name: dob_years_category, dtype: int64

**Промежуточный вывод** Данные в столбце dob_years разделены на 3 категории для большей наглядности. Полученные категории присвоены каждому клиенту, результат записан в новый столбец dob_years_category.

## Категоризация данных столбца children

In [30]:
# Создаем функцию категоризации по количеству детей и записи категорий в новый столбец children_category

def children_category(children):
    
    if  children  == 0:
                return 'Детей нет'
            
    if  children == 1:
            return '1 ребенок'

    return 'Более 1 ребенка'

In [31]:
# Вызываем функцию категоризации и выводим результат

data['children_category'] = data['children'].apply(children_category)
data['children_category'].value_counts()

Детей нет          14149
1 ребенок           4865
Более 1 ребенка     2511
Name: children_category, dtype: int64

**Промежуточный вывод** Данные в столбце children разделены на 3 категории для большей наглядности. Полученные категории присвоены каждому клиенту, результат записан в новый столбец children_category.

# Проверка гипотез

## Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
# Подсчитываем количество клиентов по количеству детей в общем и для случаев наличия просрочки по выплате. 
# Далее поделим второе на первое для получения представления о процентном соотношении двух величин 

data.groupby('children_category').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                 
                   count   sum <lambda_0>
children_category                        
1 ребенок           4865   445      9.15%
Более 1 ребенка     2511   233      9.28%
Детей нет          14149  1063      7.51%

**Вывод**

На данном этапе проекта была проверена гипотеза о том, что наличие просрочки по выплате кредита зависят от наличия у заемщика детей. 
Результаты позволяют сделать следующие выводы:
1. клиенты, у которых нет детей, имеют задолженность по кредиту в 7,51% случаев от общего количества клиентов без детей;
2. заемщики с одним ребенком имеют долг по кредиту в 9,15% случаев от общего количества таковых;
3. с увеличением количества детей показатель долга по кредиту ухудшается еще и становится равным 9,28% от общего количества многодетных клиентов.

Данные факты позволяют сделать предположение о снижении степени надежности клиента по мере роста его семьи. Если детей у заемщика нет, то он чаще платит кредит вовремя.

## Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
# Подсчитываем количество клиентов по их семейному статусу в общем и для случаев наличия просрочки по выплате. 
# Далее поделим второе на первое для получения представления о процентном соотношении двух величин 

data.groupby('family_status').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
                       count  sum <lambda_0>
family_status                               
Не женат / не замужем   2813  274      9.74%
в разводе               1195   85      7.11%
вдовец / вдова           960   63      6.56%
гражданский брак        4177  388      9.29%
женат / замужем        12380  931      7.52%

   **Вывод** 
   
На данном этапе нами проверялась гипотеза о том, что между семейным положением клиента и его возможностью вовремя платить по кредиту есть зависимость. Полученные результаты позволяют сформировать следующие выводы:

1. абсолютное первенство по кредитной недисциплинированности с 9,74 % уверенно держат холостяки;
2. адепты гражданского брака имеют просрочки в 9,29% случаев и получают 2 место по долгам;
3. сторонники официального брака не выплачивают кредит вовремя в 7,52% случаев;
4. 4 и 5 места занимают клиенты в разводе (7,11%) и заемщики, потерявшие свою половину (6,56%).

Полученные данные позволяют предположить, что по мере наращивания жизненного опыта клиенты банка становятся более дисциплинированными заемщиками и стараются вовремя покрывать выплаты по кредитам.
</div>    
</div>

## Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
# # Получаем процентное отношение людей с долгом по кредиту к количеству клиентов по категориям их доходов и выводим результаты

data.groupby('income_category').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                 
                   count   sum <lambda_0>
income_category                          
До 100 тыс          4463   354      7.93%
От 100 до 200 тыс  11807  1021      8.65%
От 200 и выше тыс   5255   366      6.96%

**Вывод** В данном разделе нами была проверена теория о том, что "надежность" заемщика зависит от уровня его доходов. Результаты расчета позволяют нам сделать следующие выводы:
1. линейной зависимости данного параметра от зарплаты клиента нет;
2. заемщик с доходом до 100 тысяч рублей имеет задолженность по кредиту реже, чем клиент с доходом от 100 до 200 тысяч рублей (7,93 % против 8,65 %). Однако при доходе от 200 тысяч рублей количество должников резко падает, составляя всего 6,96 %.

## Как разные цели кредита влияют на его возврат в срок?

In [35]:
# Cчитаем процентное соотношение и выводим результаты

data.groupby('purpose_category').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
                    count  sum <lambda_0>
purpose_category                         
автомобиль           4315  403      9.34%
жилье/недвижимость  10840  782      7.21%
образование          4022  370       9.2%
свадьба              2348  186      7.92%

**Вывод** На данном этапе проекта была проверена гипотеза о зависимости своевременности выплаты кредита от цели его взятия. Анализ результатов позволяет утверждать следующее:
1. чаще всего клиенты имеют задолженность по кредитам, если берут их для осуществления каких-либо действий с автомобилем (9,3% "должников") и для получения образования (9,2% "должников");
2. клиенты, решившие взять кредит на свадьбу занимают 3 место среди должников и не платят вовремя в 7,9% случаев;
3. самыми дисциплинированными оказываются желающие решить проблемы с жильем или другие вопросы с недвижимостью - среди них всего 7,2% должников.

# Общий вывод по проекту
В данном проекте при использовании библиотеки Pandas пошагово отработаны следующие этапы:
1. загружен исходный набор данных, получена общая информация о его наполнении;
2. осуществлена проверка датасета на наличие пропусков, артефактов, дубликатов, неверного регистра в составе данных; обнаруженные "аномалии" исправлены таким образом, чтобы имеющиеся в таблице сведения можно было использовать для проведения анализа и нахождения на его основе определенных закономерностей в дальнейшем;
3. написана функция, позволяющая лемматизировать данные о цели взятия кредита. В результате стало возможным разделить клиентов банка на категории по целям использования занятых денег. Дополнительно категоризированы содержания столбцов children, dob_years, total_income;
4. проверены следующие гипотезы:
   - о зависимости числа должников среди клиентов от наличия у них детей - (7,52% должников без детей, 9,15% - с одним ребенком, 9,28 % - более одного ребенка);
   - от иъх семейного положения ( 9,74 % должников - холостяков, в гражданском браке -  9,29 %,  7,52 % - в официальном браке, в разводе - 7,11 %, вдовы - 6,56 %); 
   - уровня дохода (до 100 тыс - 7.93% должников, от 100 до 200 тыс - 8.58%, от 200 и выше тыс - 7.07%)
   - целей взятия займа (автомобиль - 9.34%, жилье/недвижимость - 7.21%, образование - 9.2%, свадьба - 7.92%). 

Полученные числовые значения позволяют оценить, какая из рассматриваемых категорий является более, а какая менее надежной при осуществлении выплат кредитной организации.